In [1]:
import respy as rp
import numpy as np
from estimagic.optimization.optimize import maximize

### Working step 1: Model estimation via Maximum Likelihood

* We estimate a model by maximum likelihood using all true values as the starting point. Note that this model does not include $\eta$ and none of the values is fixed (different to Working step 3). We use a gradient-free optimizer.

Simulate the model at starting values.

In [2]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
simulate = rp.get_simulate_func(params, options)
df = simulate(params)

Get the criterion function and output the value of the MLE at given parameter values.

In [3]:
crit_func = rp.get_crit_func(params, options, df)
crit_func(params)

-380.4868525056946

In [4]:
# Note for RS: maxfun is for scipy optimizers
# Note for RS: maxeval is for nlopt optimizer
#algo_options = {"maxeval": 100}
#algo_options={"maxfun": 1}
#algo_name = "nlopt_bobyqa" (gradient-free)
#algo_name = "scipy_L-BFGS-B" (gradient-based)

In [5]:
# Assemble the parameter constraints for kw_94_one
# Note: # constr = rp.get_parameter_constraints("kw_94_one") # is not working
constr = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"},
    {"loc": "lagged_choice_1_edu", "type": "fixed"},
    {"loc": "initial_exp_edu_10", "type": "fixed"},
    {"loc": "maximum_exp", "type": "fixed"},
]

In [6]:
constr

[{'loc': 'shocks_sdcorr', 'type': 'sdcorr'},
 {'loc': 'lagged_choice_1_edu', 'type': 'fixed'},
 {'loc': 'initial_exp_edu_10', 'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'}]

In [7]:
results, params_rslt = maximize(
    crit_func, 
    params, 
    "nlopt_bobyqa", 
    algo_options={"maxeval": 2},  # for the first tries we set the evaluations to a low number
    constraints=constr,
    dashboard=False)

In [8]:
#params_rslt

### Working step 2: Ambiguity perturbation

We will disturb $\eta$ and subsequently estimate it to check whether the same values come out again.

More specifically:
* we will first set $\eta$ so the values presented at p.17 in the paper, i.e. baseline (0.01) and high (0.02)
* we will simulate the model with the set values
* we will perturb our starting values in the estimation and analyze whether they return to their initial values

In [9]:
# Check the parameter combinations
eta_perturbed = {
    "true": [],
    "estimated": [],
    "yos": []
}

eta_perturbed = {
    "true": [],
    "estimated": []
}

In [10]:
constr = [{"query": "category != 'eta'", "type": "fixed"}]

In [11]:
constr

[{'query': "category != 'eta'", 'type': 'fixed'}]

In [12]:
eta_baseline = 0.01
eta_high = 0.02
# To do: in the end we will have a wrapper that works for simulation with any given eta

In [13]:
# Decide for an eta
params, options = rp.get_example_model("kw_94_one", with_data=False)
params.loc[("eta", "eta"), :] = (eta_baseline, "eta value " + str(eta_baseline))
simulate = rp.get_simulate_func(params, options)

In [14]:
for eta in np.linspace(0, 0.25, 11):
    
    #params, options = rp.get_example_model("kw_94_one", with_data=False)
    #simulate = rp.get_simulate_func(params, options)
    
    #params.loc[("eta", "eta"), :] = (eta, "iteration with " + str(eta))
    params.loc[("eta", "eta"), :] = eta
    #df = simulate(params) # We want only to change the starting values, but not the simulated "data"
    #stat = df.groupby("Identifier")["Experience_Edu"].max().mean()
    
    #params["group"] = params.index.get_level_values('category')
    # need above only if we want dashboard output
    
    crit_func = rp.get_crit_func(params, options, df)
    # crit_func(params) # no need to output the value here
    # constr = rp.get_parameter_constraints("kw_94_one")
    # just take constraints from above with all other except eta fixed

    results, params = maximize(
        crit_func, 
        params,
        "nlopt_bobyqa",
        algo_options={"maxeval": 2},
        # In the "real world simulation" we will set this to a much higher number
        constraints=constr,
        dashboard=False
    )
    
    eta_perturbed["true"].append(eta_baseline)
    eta_perturbed["estimated"].append(params.loc["eta", "value"][0])
    #eta_perturbed["yos"].append(stat)
    
    # To do: write a wrapper for this thing

ValueError: Column names starting with '_internal' and as well as any other of the following columns are not allowed in params:
['_fixed_value', '_is_fixed_to_value', '_is_fixed_to_other'].This is violated for:
['_fixed_value', '_is_fixed_to_value', '_is_fixed_to_other', '_internal_lower', '_internal_upper', '_internal_free', '_internal_fixed_value'].

### Working step 3: Risk-only model

We estimate a model with *risk-only*. This means we have $\eta = 0$ fixed. 
We want to see how different the likelihood is at the estimated parameter values.
We treat the risk-parameter as only unknown parameter.

In [ ]:
params.loc[("eta", "eta"), :] = (0, "Ambiguity fixed at zero")
constr = {"query": "category != delta", "type": "fixed"}

In [ ]:
crit_func = rp.get_crit_func(params, options, df)
crit_func(params)

In [ ]:
results, params_rslt = maximize(
    crit_func, params,
    "nlopt_bobyqa",
    algo_options={"maxeval": 2},
    constraints=constr,
    dashboard=False
)

In [ ]:
params_rslt

### Summary of working steps

* We estimated a model by maximum likelihood using all true values as the starting point.

* We estimated a model with only $\eta$ perturbed.

* We estimated a model with risk-only, $\eta = 0$ fixed, to see how different the likelihood is at the estimated parameter values.